TO DO:
- Add option to force players into the team, i.e., optimize given certain player are already locked in

In [1]:
# previous gameweek
gameweek=25

In [2]:
import pandas as pd
import numpy as np
import json
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [3]:
# fetch player_dict 
with open('../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)

# Solver definition

In [4]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES

    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    captaincy_variables = [LpVariable(name=f'cap_{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'price'] for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'position']=='GK' else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'position']=='DEF' else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'position']=='MID' else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'position']=='FWD' else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # select 1 captain
    model += lpSum(captaincy_variables) == 1
    # captain must be one of the 11 players in the team
    for i in range(0,len(variables)):
        model += captaincy_variables[i] <= variables[i]
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3

    # OBJECTIVE
    # the second part doubles the captain's points
    model += np.dot(expected_points,variables) + np.dot(expected_points,captaincy_variables)

    # SOLVE OPTIMIZATION

    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [str(var) for var in model.variables() if var.value()==1]
    captain = [player for player in players if 'cap' in player]
    captain = int(captain[0].replace('cap_', ''))
    players = [int(player) for player in players if 'cap' not in player]
    dream_team = df.loc[players]
    cost = (dream_team['price']).sum()
    exp_points = dream_team[expected_column].sum() + df.loc[captain, expected_column]
    print(f'Cost: {cost}')
    print(f'Expected points per week: {exp_points}')
    
    display(dream_team[['position','name', 'team', 'price',expected_column]].sort_values('position'))
    
    return players

# Data

In [5]:
#projections = pd.read_csv(f'../point_projections/xP_projections_week{gameweek}.csv', index_col=0)
projections = pd.read_csv(f'../point_projections/collective_xP_projections_week{gameweek}.csv', index_col=0)
projections = projections.rename(columns={'xP':'projected_points'})
projections.head(5)

name     team        opp_team  was_home  gameweek  \
0  Granit Xhaka  Arsenal     Bournemouth         1        26   
1  Granit Xhaka  Arsenal          Fulham         0        27   
2  Granit Xhaka  Arsenal  Crystal Palace         1        28   
3  Granit Xhaka  Arsenal    Leeds United         1        29   
4  Granit Xhaka  Arsenal       Liverpool         0        30   

   projected_points       std  
0          3.605445  0.220236  
1          3.195131  0.224736  
2          3.579107  0.198305  
3          3.320841  0.155277  
4          2.510845  0.133115

In [6]:
# decay for future projected points to model uncertainty increasing over time
decay_coeff = 0.875
projections['projected_points_with_decay'] = ( projections['projected_points']*decay_coeff
                                              **(projections['gameweek']-(gameweek+1)) )
projections.head(5)

name     team        opp_team  was_home  gameweek  \
0  Granit Xhaka  Arsenal     Bournemouth         1        26   
1  Granit Xhaka  Arsenal          Fulham         0        27   
2  Granit Xhaka  Arsenal  Crystal Palace         1        28   
3  Granit Xhaka  Arsenal    Leeds United         1        29   
4  Granit Xhaka  Arsenal       Liverpool         0        30   

   projected_points       std  projected_points_with_decay  
0          3.605445  0.220236                     3.605445  
1          3.195131  0.224736                     2.795739  
2          3.579107  0.198305                     2.740254  
3          3.320841  0.155277                     2.224704  
4          2.510845  0.133115                     1.471811

In [7]:
fpl_df = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv', index_col=0)
fpl_df['name'] = fpl_df['name'].map(player_name_dict).fillna(fpl_df['name'])
fpl_df.head(5)

chance_of_playing_next_round  chance_of_playing_this_round    code  \
0                           NaN                           NaN   84450   
1                           0.0                           0.0  153256   
2                           NaN                           NaN  156074   
3                         100.0                          75.0  167199   
4                         100.0                         100.0  184029   

   cost_change_event  cost_change_event_fall  cost_change_start  \
0                  0                       0                 -1   
1                  0                       0                 -4   
2                  0                       0                 -3   
3                  0                       0                 -3   
4                  0                       0                  5   

   cost_change_start_fall  dreamteam_count  element_type  ep_next  ...  \
0                       1                1             3      3.3  ...   
1                       4                1             3      0.0  ...   
2                       3                0             2      1.2  ...   
3                       3                0             3      2.0  ...   
4                      -5                4             3      5.2  ...   

    season            name position  GW  value    selected transfers_balance  \
0  2022-23    Granit Xhaka      MID  25    4.9   303765.39             -7902   
1  2022-23  Mohamed Elneny      MID  25    4.1    67503.42             -1275   
2  2022-23     Rob Holding      DEF  25    4.2    11250.57               -54   
3  2022-23   Thomas Partey      MID  25    4.7    45002.28              -348   
4  2022-23  Martin degaard      MID  25    7.0  4308968.31            -29215   

  opp_team_name  kickoff_time was_home  
0           NaN           NaN      1.0  
1           NaN           NaN      1.0  
2           NaN           NaN      1.0  
3           NaN           NaN      1.0  
4           NaN           NaN      1.0  

[5 rows x 98 columns]

In [8]:
# choose time horizon for optimization
horizon = 10

# create dataframe to be given as input for optimization
df = projections[projections['gameweek']<=gameweek+horizon].groupby(by=['name']).mean()\
                                                        [['projected_points','projected_points_with_decay', 'std']] 
#df = df.to_frame()
df = df.reset_index()
df = df.rename(columns={'projected_points':'projected_points_per_game'})
#df['team'] = [fpl_df.loc[fpl_df['name']==name, 'team'].values[0] for name in df['name']]
df['price'] = [fpl_df.loc[fpl_df['name']==name, 'value'].values[0] for name in df['name']]
df['position'] = [fpl_df.loc[fpl_df['name']==name, 'element_type'].values[0] for name in df['name']]
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
df['position'] = df['position'].map(position_dict)
df['position'] = pd.Categorical(df['position'], ['GK','DEF','MID','FWD'])
df['team'] = [projections.loc[projections['name']==name, 'team'].head(1).values[0] for name in df['name']]
df['minutes'] = [fpl_df.loc[fpl_df['name']==name, 'minutes'].values[0] for name in df['name']]
# re-order columns
df = df[['name', 'team', 'position', 'price', 'minutes', 'projected_points_per_game', 'projected_points_with_decay', 'std']]
df.head(5)

/var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/ipykernel_76602/2245439599.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = projections[projections['gameweek']<=gameweek+horizon].groupby(by=['name']).mean()\


name            team position  price  minutes  \
0     Aaron Cresswell        West Ham      DEF    4.7     1578   
1        Aaron Hickey       Brentford      DEF    4.9      815   
2      Aaron Ramsdale         Arsenal       GK    4.9     2250   
3   Aaron Wan-Bissaka  Manchester Utd      DEF    4.3      646   
4  Abdoulaye Doucoure         Everton      MID    5.3      660   

   projected_points_per_game  projected_points_with_decay       std  
0                   2.982367                     1.730514  0.109547  
1                   2.543225                     1.598735  0.086609  
2                   3.458309                     2.109378  0.167265  
3                   2.840811                     1.643773  0.115318  
4                   2.248236                     1.329925  0.131267

In [9]:
# remove players who do not have enough minutes played
cutoff = 450
df = df[df['minutes']>=cutoff]

# edit some player prices according to my selling price
df.loc[df['name']=='Erling Haaland', 'price'] = 11.8
df.loc[df['name']=='Martinelli', 'price'] = 6.3

# Optimization

In [10]:
team_value = 101.7
money_in_the_bank = 0.6
allowed_bench_value = 17.5
max_price = team_value + money_in_the_bank - allowed_bench_value

## Without decay

In [11]:
players = optimize(df, max_price, 'projected_points_per_game')

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/91e01f958ce24c5c839ff68ad77dc08a-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/91e01f958ce24c5c839ff68ad77dc08a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 333 COLUMNS
At line 4367 RHS
At line 4696 BOUNDS
At line 5295 ENDATA
Problem MODEL has 328 rows, 598 columns and 2239 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 59.3616 - 0.00 seconds
Cgl0004I processed model has 326 rows, 598 columns (598 integer (598 of which binary)) and 2093 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.285714
Cbc0038I Pass   1: suminf.    0.05063 (2) obj. -59.3079 iterations 6
Cbc0038I Solution found of -56.0714
Cbc0038I Before mini bran

position                    name             team  price  \
118       GK              David Raya        Brentford    4.7   
265      DEF             John Stones  Manchester City    5.4   
297      DEF         Kieran Trippier    Newcastle Utd    6.1   
520      DEF  Trent Alexander-Arnold        Liverpool    7.3   
354      MID          Martin degaard          Arsenal    7.0   
379      MID           Mohamed Salah        Liverpool   12.7   
447      MID            Riyad Mahrez  Manchester City    7.7   
69       MID             Bukayo Saka          Arsenal    8.3   
14       FWD     Aleksandar Mitrovic           Fulham    7.0   
185      FWD              Harry Kane        Tottenham   11.8   
206      FWD              Ivan Toney        Brentford    7.7   

     projected_points_per_game  
118                   3.692618  
265                   3.909687  
297                   4.504202  
520                   4.140502  
354                   4.999818  
379                   6.528418  
447                   4.652731  
69                    4.979669  
14                    4.902477  
185                   5.664962  
206                   4.774251

In [12]:
ban = []
# ban some cheap keepers in good teams who don't actually play
cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
                 'Karl Darlow', 'Darren Randolph']
cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
ban = ban + list(cheap_keepers_ix)
# ban given players
ban_field = ['Marcos Alonso', 'Ben Chilwell', 'Riyad Mahrez', 'Darwin Nunez', 'Aymeric Laporte', 'Emil Krafth', 
             'Sergio Gomez', 'Nathaniel Phillips']
ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [13]:
players = optimize(my_df, max_price, 'projected_points_per_game')

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/3597031289b64eda9f26d5b5f4986aaa-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/3597031289b64eda9f26d5b5f4986aaa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 331 COLUMNS
At line 4338 RHS
At line 4665 BOUNDS
At line 5260 ENDATA
Problem MODEL has 326 rows, 594 columns and 2224 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 59.3472 - 0.00 seconds
Cgl0004I processed model has 324 rows, 594 columns (594 integer (594 of which binary)) and 2079 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.393443
Cbc0038I Solution found of -59.1795
Cbc0038I Before mini branch and bound, 592 integers at bound fixed and 0 continuous
Cbc0038I F

position                    name           team  price  \
118       GK              David Raya      Brentford    4.7   
29       DEF        Andrew Robertson      Liverpool    6.8   
297      DEF         Kieran Trippier  Newcastle Utd    6.1   
520      DEF  Trent Alexander-Arnold      Liverpool    7.3   
354      MID          Martin degaard        Arsenal    7.0   
355      MID              Martinelli        Arsenal    6.3   
379      MID           Mohamed Salah      Liverpool   12.7   
69       MID             Bukayo Saka        Arsenal    8.3   
14       FWD     Aleksandar Mitrovic         Fulham    7.0   
185      FWD              Harry Kane      Tottenham   11.8   
206      FWD              Ivan Toney      Brentford    7.7   

     projected_points_per_game  
118                   3.692618  
29                    4.111096  
297                   4.504202  
520                   4.140502  
354                   4.999818  
355                   4.440250  
379                   6.528418  
69                    4.979669  
14                    4.902477  
185                   5.664962  
206                   4.774251

## With decay

In [11]:
#players = optimize(df, max_price, 'projected_points_with_decay')

In [12]:
ban = []
# ban some cheap keepers in good teams who don't actually play
#cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
#                 'Karl Darlow', 'Darren Randolph', 'Alphonse Areola', 'Asmir Begovic', 
#                'Sam Johnstone', 'Jack Butland']
#cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
#ban = ban + list(cheap_keepers_ix)
# ban given players
#ban_field = ['Marcos Alonso', 'Riyad Mahrez', 'Nathaniel Phillips', 'Alexander Isak', 'Stefan Bajcetic',]
#ban_field_ix = df[df['name'].isin(ban_field)].index
#ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [13]:
players = optimize(my_df, max_price, 'projected_points_with_decay')

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/85ca9ed321cd4b36b4b7fa175783c26d-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/85ca9ed321cd4b36b4b7fa175783c26d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 356 COLUMNS
At line 4707 RHS
At line 5059 BOUNDS
At line 5704 ENDATA
Problem MODEL has 351 rows, 644 columns and 2418 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 34.8264 - 0.01 seconds
Cgl0004I processed model has 349 rows, 644 columns (644 integer (644 of which binary)) and 2254 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.508475
Cbc0038I Pass   1: suminf.    0.36145 (2) obj. -34.6471 iterations 5
Cbc0038I Solution found of -33.3191
Cbc0038I Before mini bran

position                    name             team  price  \
119       GK              David Raya        Brentford    4.8   
300      DEF         Kieran Trippier    Newcastle Utd    6.0   
45       DEF         Aymeric Laporte  Manchester City    5.7   
529      DEF  Trent Alexander-Arnold        Liverpool    7.3   
360      MID          Martin degaard          Arsenal    7.0   
386      MID           Mohamed Salah        Liverpool   12.7   
498      MID             Solly March         Brighton    5.1   
70       MID             Bukayo Saka          Arsenal    8.5   
175      FWD           Gabriel Jesus          Arsenal    7.9   
187      FWD              Harry Kane        Tottenham   11.7   
208      FWD              Ivan Toney        Brentford    7.6   

     projected_points_with_decay  
119                     2.299865  
300                     2.857040  
45                      2.278872  
529                     2.358687  
360                     3.107672  
386                     3.437400  
498                     2.996761  
70                      2.902839  
175                     2.724801  
187                     3.305258  
208                     3.015145

# Sensitivity analysis

Here we add small random Gaussian noise to each players' projected points and run the optimization with these new estimates. We repeat the process n times and keep a list of how many times each player appeared in the optimal team.

In [14]:
ban = []
# ban some cheap keepers in good teams who don't actually play
#cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
#                 'Karl Darlow', 'Darren Randolph', 'Alphonse Areola', 'Asmir Begovic', 
#                'Sam Johnstone', 'Jack Butland']
#cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
#ban = ban + list(cheap_keepers_ix)
# ban given players
#ban_field = ['Kyle Walker']
#ban_field_ix = df[df['name'].isin(ban_field)].index
#ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [15]:
metric = 'projected_points_with_decay'
# set the standard deviation of the Gaussian noise to be a fraction of each players' projected points
#sigma_factor = 0.1
simulation_rounds = 100
results = []
for i in range(0,simulation_rounds):
    df_with_noise = my_df.copy()
    #df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise[metric]*sigma_factor)
    df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise['std'])
    players = optimize(df_with_noise, max_price, metric)
    results.append(players)

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/2e8789aab2ae4a7eb36f0b0a85ad8f15-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/2e8789aab2ae4a7eb36f0b0a85ad8f15-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 356 COLUMNS
At line 4707 RHS
At line 5059 BOUNDS
At line 5704 ENDATA
Problem MODEL has 351 rows, 644 columns and 2418 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 36.6125 - 0.00 seconds
Cgl0004I processed model has 349 rows, 644 columns (644 integer (644 of which binary)) and 2254 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -36.6125
Cbc0038I Before mini branch and bound, 644 integers at bound fixed and 0 continuous
Cbc0038I Mini bra

position                    name             team  price  \
119       GK              David Raya        Brentford    4.8   
168      DEF            Fabian Schar    Newcastle Utd    5.1   
319      DEF              Lewis Dunk         Brighton    4.7   
529      DEF  Trent Alexander-Arnold        Liverpool    7.3   
360      MID          Martin degaard          Arsenal    7.0   
386      MID           Mohamed Salah        Liverpool   12.7   
455      MID            Riyad Mahrez  Manchester City    7.8   
498      MID             Solly March         Brighton    5.1   
67       MID         Bruno Fernandes   Manchester Utd    9.8   
162      FWD          Erling Haaland  Manchester City   11.8   
208      FWD              Ivan Toney        Brentford    7.6   

     projected_points_with_decay  
119                     2.346453  
168                     2.351252  
319                     2.488585  
529                     2.448172  
360                     4.049825  
386                     3.292872  
455                     2.959123  
498                     3.127866  
67                      2.903698  
162                     3.556551  
208                     3.038299

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/597f7e23d32a46ffb718797a620ec7b7-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/597f7e23d32a46ffb718797a620ec7b7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 356 COLUMNS
At line 4707 RHS
At line 5059 BOUNDS
At line 5704 ENDATA
Problem MODEL has 351 rows, 644 columns and 2418 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 36.1505 - 0.01 seconds
Cgl0004I processed model has 349 rows, 644 columns (644 integer (644 of which binary)) and 2254 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.142857
Cbc0038I Pass   1: suminf.    0.06818 (2) obj. -36.1048 iterations 9
Cbc0038I Solution found of -33.9869
Cbc0038I Before mini bran

position                 name             team  price  \
119       GK           David Raya        Brentford    4.8   
161      DEF            Eric Dier        Tottenham    5.1   
417      DEF  Oleksandr Zinchenko          Arsenal    5.1   
55       DEF    Benoit Badiashile          Chelsea    5.0   
294      MID      Kevin De Bruyne  Manchester City   12.2   
360      MID       Martin degaard          Arsenal    7.0   
386      MID        Mohamed Salah        Liverpool   12.7   
498      MID          Solly March         Brighton    5.1   
70       MID          Bukayo Saka          Arsenal    8.5   
187      FWD           Harry Kane        Tottenham   11.7   
208      FWD           Ivan Toney        Brentford    7.6   

     projected_points_with_decay  
119                     2.386817  
161                     2.212554  
417                     2.431168  
55                      2.560602  
294                     3.381887  
360                     3.164768  
386                     3.526651  
498                     3.106654  
70                      3.561311  
187                     3.029739  
208                     3.071638

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/4abf55ba5c7a42df863f6f78e3f3566e-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/4abf55ba5c7a42df863f6f78e3f3566e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 356 COLUMNS
At line 4707 RHS
At line 5059 BOUNDS
At line 5704 ENDATA
Problem MODEL has 351 rows, 644 columns and 2418 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 38.3159 - 0.01 seconds
Cgl0004I processed model has 349 rows, 644 columns (644 integer (644 of which binary)) and 2254 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.193548
Cbc0038I Solution found of -38.283
Cbc0038I Before mini branch and bound, 642 integers at bound fixed and 0 continuous
Cbc0038I Fu

...

position                 name             team  price  \
148       GK              Ederson  Manchester City    5.4   
165      DEF        Ethan Pinnock        Brentford    4.4   
417      DEF  Oleksandr Zinchenko          Arsenal    5.1   
45       DEF      Aymeric Laporte  Manchester City    5.7   
360      MID       Martin degaard          Arsenal    7.0   
381      MID       Miguel Almiron    Newcastle Utd    5.5   
386      MID        Mohamed Salah        Liverpool   12.7   
498      MID          Solly March         Brighton    5.1   
67       MID      Bruno Fernandes   Manchester Utd    9.8   
162      FWD       Erling Haaland  Manchester City   11.8   
187      FWD           Harry Kane        Tottenham   11.7   

     projected_points_with_decay  
148                     2.306590  
165                     2.176092  
417                     2.366836  
45                      2.237799  
360                     3.337616  
381                     2.917559  
386                     3.336385  
498                     2.904060  
67                      3.417298  
162                     3.528482  
187                     3.399595

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/af399589d9c94d4dac354d78ee15059c-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/af399589d9c94d4dac354d78ee15059c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 356 COLUMNS
At line 4707 RHS
At line 5059 BOUNDS
At line 5704 ENDATA
Problem MODEL has 351 rows, 644 columns and 2418 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 36.6251 - 0.00 seconds
Cgl0004I processed model has 349 rows, 644 columns (644 integer (644 of which binary)) and 2254 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -36.6251
Cbc0038I Before mini branch and bound, 644 integers at bound fixed and 0 continuous
Cbc0038I Mini bra

position               name             team  price  \
119       GK         David Raya        Brentford    4.8   
143      DEF    Duje Caleta-Car      Southampton    4.5   
214      DEF      Jack Stephens      Bournemouth    4.3   
55       DEF  Benoit Badiashile          Chelsea    5.0   
294      MID    Kevin De Bruyne  Manchester City   12.2   
360      MID     Martin degaard          Arsenal    7.0   
386      MID      Mohamed Salah        Liverpool   12.7   
498      MID        Solly March         Brighton    5.1   
117      FWD       Darwin Nunez        Liverpool    8.8   
187      FWD         Harry Kane        Tottenham   11.7   
208      FWD         Ivan Toney        Brentford    7.6   

     projected_points_with_decay  
119                     2.369177  
143                     2.894503  
214                     2.742810  
55                      2.417071  
294                     3.584119  
360                     3.506624  
386                     3.149789  
498                     3.050379  
117                     3.097168  
187                     3.021802  
208                     3.207516

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/923ec421ffa74da99d0c368c53fd1da8-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/923ec421ffa74da99d0c368c53fd1da8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 356 COLUMNS
At line 4707 RHS
At line 5059 BOUNDS
At line 5704 ENDATA
Problem MODEL has 351 rows, 644 columns and 2418 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 35.68 - 0.00 seconds
Cgl0004I processed model has 349 rows, 644 columns (644 integer (644 of which binary)) and 2254 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.861538
Cbc0038I Solution found of -35.6793
Cbc0038I Before mini branch and bound, 642 integers at bound fixed and 0 continuous
Cbc0038I Ful

position                    name           team  price  \
119       GK              David Raya      Brentford    4.8   
29       DEF        Andrew Robertson      Liverpool    6.8   
529      DEF  Trent Alexander-Arnold      Liverpool    7.3   
55       DEF       Benoit Badiashile        Chelsea    5.0   
360      MID          Martin degaard        Arsenal    7.0   
381      MID          Miguel Almiron  Newcastle Utd    5.5   
386      MID           Mohamed Salah      Liverpool   12.7   
498      MID             Solly March       Brighton    5.1   
70       MID             Bukayo Saka        Arsenal    8.5   
187      FWD              Harry Kane      Tottenham   11.7   
208      FWD              Ivan Toney      Brentford    7.6   

     projected_points_with_decay  
119                     2.426891  
29                      2.543134  
529                     2.588031  
55                      2.493454  
360                     3.065932  
381                     3.046896  
386                     3.742312  
498                     2.760461  
70                      3.495263  
187                     2.869622  
208                     2.905023

In [16]:
my_df['dream_team_appearances'] = [sum(team.count(player_id) for team in results) for player_id in my_df.index]
my_df.head(5)

name            team position  price  minutes  \
0     Aaron Cresswell        West Ham      DEF    4.7     1578   
1        Aaron Hickey       Brentford      DEF    4.9      815   
2      Aaron Ramsdale         Arsenal       GK    4.9     2250   
3   Aaron Wan-Bissaka  Manchester Utd      DEF    4.3      646   
4  Abdoulaye Doucoure         Everton      MID    5.3      660   

   projected_points_per_game  projected_points_with_decay       std  \
0                   2.982367                     1.730514  0.109547   
1                   2.543225                     1.598735  0.086609   
2                   3.458309                     2.109378  0.167265   
3                   2.840811                     1.643773  0.115318   
4                   2.248236                     1.329925  0.131267   

   dream_team_appearances  
0                       0  
1                       0  
2                       7  
3                       0  
4                       0

In [17]:
print('Most appearances, all players:')
display(my_df.sort_values(by='dream_team_appearances', ascending=False).head(20))

Most appearances, all players:


name             team position  price  minutes  \
208              Ivan Toney        Brentford      FWD    7.6     1881   
498             Solly March         Brighton      MID    5.1     1900   
187              Harry Kane        Tottenham      FWD   11.7     2236   
386           Mohamed Salah        Liverpool      MID   12.7     2136   
360          Martin degaard          Arsenal      MID    7.0     2041   
300         Kieran Trippier    Newcastle Utd      DEF    6.0     2017   
70              Bukayo Saka          Arsenal      MID    8.5     2155   
119              David Raya        Brentford       GK    4.8     2070   
294         Kevin De Bruyne  Manchester City      MID   12.2     1860   
162          Erling Haaland  Manchester City      FWD   11.8     1931   
67          Bruno Fernandes   Manchester Utd      MID    9.8     2069   
143         Duje Caleta-Car      Southampton      DEF    4.5      713   
14      Aleksandar Mitrovic           Fulham      FWD    6.8     1636   
268             John Stones  Manchester City      DEF    5.4     1180   
529  Trent Alexander-Arnold        Liverpool      DEF    7.3     1778   
455            Riyad Mahrez  Manchester City      MID    7.8     1055   
381          Miguel Almiron    Newcastle Utd      MID    5.5     1879   
45          Aymeric Laporte  Manchester City      DEF    5.7      555   
547           Wilfried Zaha   Crystal Palace      MID    7.2     1661   
29         Andrew Robertson        Liverpool      DEF    6.8     1590   

     projected_points_per_game  projected_points_with_decay       std  \
208                   4.780048                     3.015145  0.205603   
498                   4.959797                     2.996761  0.391536   
187                   5.445996                     3.305258  0.354251   
386                   5.910157                     3.437400  0.661048   
360                   5.117631                     3.107672  0.348813   
300                   4.834021                     2.857040  0.819507   
70                    4.776881                     2.902839  0.408151   
119                   3.657949                     2.299865  0.230263   
294                   5.449208                     3.066999  0.585238   
162                   5.038960                     2.852106  0.654890   
67                    4.838812                     2.798462  0.591538   
143                   2.709165                     1.661914  0.883469   
14                    4.733273                     2.650365  0.339496   
268                   3.906857                     2.225572  0.178296   
529                   4.080194                     2.358687  0.193538   
455                   4.475142                     2.550843  0.451207   
381                   4.268262                     2.523532  0.294568   
45                    4.002408                     2.278872  0.140618   
547                   4.235273                     2.578925  0.322303   
29                    3.962584                     2.292681  0.209754   

     dream_team_appearances  
208                      76  
498                      72  
187                      72  
386                      68  
360                      66  
300                      56  
70                       52  
119                      45  
294                      43  
162                      38  
67                       30  
143                      27  
14                       26  
268                      25  
529                      24  
455                      24  
381                      23  
45                       23  
547                      21  
29                       20

In [18]:
print('GOALKEEPERS')
display(my_df[my_df['position']=='GK'].sort_values(by='dream_team_appearances', ascending=False).head(10))

GOALKEEPERS


name             team position  price  minutes  \
119         David Raya        Brentford       GK    4.8     2070   
198        Hugo Lloris        Tottenham       GK    5.4     1890   
148            Ederson  Manchester City       GK    5.4     2250   
2       Aaron Ramsdale          Arsenal       GK    4.9     2250   
293  Kepa Arrizabalaga          Chelsea       GK    4.7     1395   
21             Alisson        Liverpool       GK    5.4     2160   
457     Robert Sanchez         Brighton       GK    4.6     1980   
537     Vicente Guaita   Crystal Palace       GK    4.4     2160   
120       David de Gea   Manchester Utd       GK    5.0     2160   
332   Lukasz Fabianski         West Ham       GK    5.0     2031   

     projected_points_per_game  projected_points_with_decay       std  \
119                   3.657949                     2.299865  0.230263   
198                   3.552668                     2.135678  0.229504   
148                   3.787019                     2.157567  0.228948   
2                     3.458309                     2.109378  0.167265   
293                   3.503630                     2.124479  0.155217   
21                    3.557529                     2.058575  0.185813   
457                   3.226446                     1.951964  0.139907   
537                   3.206831                     1.936294  0.116409   
120                   3.358385                     1.939216  0.141171   
332                   3.364935                     1.953914  0.134849   

     dream_team_appearances  
119                      45  
198                      16  
148                      13  
2                         7  
293                       7  
21                        4  
457                       3  
537                       2  
120                       1  
332                       1

In [19]:
print('DEFENDERS')
display(my_df[my_df['position']=='DEF'].sort_values(by='dream_team_appearances', ascending=False).head(20))

DEFENDERS


name             team position  price  minutes  \
300         Kieran Trippier    Newcastle Utd      DEF    6.0     2017   
143         Duje Caleta-Car      Southampton      DEF    4.5      713   
268             John Stones  Manchester City      DEF    5.4     1180   
529  Trent Alexander-Arnold        Liverpool      DEF    7.3     1778   
45          Aymeric Laporte  Manchester City      DEF    5.7      555   
29         Andrew Robertson        Liverpool      DEF    6.8     1590   
55        Benoit Badiashile          Chelsea      DEF    5.0      450   
174      Gabriel Dos Santos          Arsenal      DEF    5.2     2250   
214           Jack Stephens      Bournemouth      DEF    4.3      498   
168            Fabian Schar    Newcastle Utd      DEF    5.1     1964   
417     Oleksandr Zinchenko          Arsenal      DEF    5.1     1432   
308             Kyle Walker  Manchester City      DEF    4.8     1072   
448             Reece James          Chelsea      DEF    5.8      849   
467              Ruben Dias  Manchester City      DEF    5.9     1187   
319              Lewis Dunk         Brighton      DEF    4.7     1980   
549          William Saliba          Arsenal      DEF    5.3     2235   
251            Joao Cancelo  Manchester City      DEF    7.1     1273   
165           Ethan Pinnock        Brentford      DEF    4.4     1350   
47             Ben Chilwell          Chelsea      DEF    5.8      617   
506             Sven Botman    Newcastle Utd      DEF    4.5     1777   

     projected_points_per_game  projected_points_with_decay       std  \
300                   4.834021                     2.857040  0.819507   
143                   2.709165                     1.661914  0.883469   
268                   3.906857                     2.225572  0.178296   
529                   4.080194                     2.358687  0.193538   
45                    4.002408                     2.278872  0.140618   
29                    3.962584                     2.292681  0.209754   
55                    3.687317                     2.234765  0.161131   
174                   3.667989                     2.234116  0.130861   
214                   2.679700                     1.573698  0.650025   
168                   3.673984                     2.162098  0.173161   
417                   3.538235                     2.160208  0.154605   
308                   3.618385                     2.065665  0.228865   
448                   3.594116                     2.176536  0.123739   
467                   3.749153                     2.138132  0.149276   
319                   3.214404                     1.946538  0.252320   
549                   3.539169                     2.160973  0.128536   
251                   3.632995                     2.073646  0.268753   
165                   3.082758                     1.943534  0.141907   
47                    3.443854                     2.084254  0.155881   
506                   3.323401                     1.950209  0.132832   

     dream_team_appearances  
300                      56  
143                      27  
268                      25  
529                      24  
45                       23  
29                       20  
55                       19  
174                      17  
214                      15  
168                      12  
417                      12  
308                      10  
448                       7  
467                       6  
319                       4  
549                       4  
251                       3  
165                       3  
47                        3  
506                       2

In [20]:
print('MIDFIELDERS')
display(my_df[my_df['position']=='MID'].sort_values(by='dream_team_appearances', ascending=False).head(20))

MIDFIELDERS


name             team position  price  minutes  \
498        Solly March         Brighton      MID    5.1     1900   
386      Mohamed Salah        Liverpool      MID   12.7     2136   
360     Martin degaard          Arsenal      MID    7.0     2041   
70         Bukayo Saka          Arsenal      MID    8.5     2155   
294    Kevin De Bruyne  Manchester City      MID   12.2     1860   
67     Bruno Fernandes   Manchester Utd      MID    9.8     2069   
455       Riyad Mahrez  Manchester City      MID    7.8     1055   
381     Miguel Almiron    Newcastle Utd      MID    5.5     1879   
547      Wilfried Zaha   Crystal Palace      MID    7.2     1661   
442    Raheem Sterling          Chelsea      MID    9.7     1268   
240       Jarrod Bowen         West Ham      MID    8.0     2151   
361         Martinelli          Arsenal      MID    6.3     2005   
95          Cody Gakpo        Liverpool      MID    7.8      490   
435         Phil Foden  Manchester City      MID    8.0     1252   
130       Dennis Praet   Leicester City      MID    4.8      460   
232  James Ward-Prowse      Southampton      MID    6.3     2160   
312   Leandro Trossard          Arsenal      MID    6.6     1604   
523        Tom Cairney           Fulham      MID    4.7      455   
211      Jack Grealish  Manchester City      MID    6.8     1342   
38              Antony   Manchester Utd      MID    7.3      910   

     projected_points_per_game  projected_points_with_decay       std  \
498                   4.959797                     2.996761  0.391536   
386                   5.910157                     3.437400  0.661048   
360                   5.117631                     3.107672  0.348813   
70                    4.776881                     2.902839  0.408151   
294                   5.449208                     3.066999  0.585238   
67                    4.838812                     2.798462  0.591538   
455                   4.475142                     2.550843  0.451207   
381                   4.268262                     2.523532  0.294568   
547                   4.235273                     2.578925  0.322303   
442                   4.490190                     2.722785  0.275241   
240                   4.537100                     2.639152  0.267155   
361                   4.310996                     2.620116  0.171968   
95                    4.146130                     2.402658  0.287996   
435                   4.417420                     2.507958  0.234789   
130                   2.682217                     1.594570  0.777358   
232                   3.625919                     2.223188  0.267570   
312                   3.902491                     2.368677  0.166998   
523                   2.573150                     1.451503  0.582164   
211                   4.246282                     2.412489  0.219380   
38                    3.976707                     2.299997  0.250811   

     dream_team_appearances  
498                      72  
386                      68  
360                      66  
70                       52  
294                      43  
67                       30  
455                      24  
381                      23  
547                      21  
442                      15  
240                      14  
361                      13  
95                        7  
435                       3  
130                       3  
232                       2  
312                       1  
523                       1  
211                       1  
38                        1

In [21]:
print('FORWARDS')
display(my_df[my_df['position']=='FWD'].sort_values(by='dream_team_appearances', ascending=False).head(20))

FORWARDS


name             team position  price  minutes  \
208            Ivan Toney        Brentford      FWD    7.6     1881   
187            Harry Kane        Tottenham      FWD   11.7     2236   
162        Erling Haaland  Manchester City      FWD   11.8     1931   
14    Aleksandar Mitrovic           Fulham      FWD    6.8     1636   
175         Gabriel Jesus          Arsenal      FWD    7.9     1220   
117          Darwin Nunez        Liverpool      FWD    8.8     1200   
419         Ollie Watkins      Aston Villa      FWD    7.3     1885   
103     Cristiano Ronaldo   Manchester Utd      FWD   10.2      519   
380       Michail Antonio         West Ham      FWD    7.0     1120   
281           Kai Havertz          Chelsea      FWD    7.6     1705   
291     Kelechi Iheanacho   Leicester City      FWD    6.2      565   
298         Kieffer Moore      Bournemouth      FWD    5.1     1113   
5          Adam Armstrong      Southampton      FWD    5.4     1060   
408           Neal Maupay          Everton      FWD    5.9      932   
244  Jean-Philippe Mateta   Crystal Palace      FWD    5.1      646   
416       Odsonne Edouard   Crystal Palace      FWD    5.1     1114   
428       Patrick Bamford     Leeds United      FWD    7.2      819   
429           Patson Daka   Leicester City      FWD    5.6      710   
446          Raul Jimenez           Wolves      FWD    6.7      495   
452           Richarlison        Tottenham      FWD    8.4      649   

     projected_points_per_game  projected_points_with_decay       std  \
208                   4.780048                     3.015145  0.205603   
187                   5.445996                     3.305258  0.354251   
162                   5.038960                     2.852106  0.654890   
14                    4.733273                     2.650365  0.339496   
175                   4.457621                     2.724801  0.224548   
117                   3.861633                     2.234032  0.345111   
419                   4.033670                     2.416553  0.205978   
103                   4.098137                     2.368969  0.174523   
380                   2.785202                     1.614074  0.129757   
281                   3.466799                     2.096478  0.136697   
291                   3.023123                     1.795561  0.137105   
298                   2.579055                     1.515603  0.173312   
5                     2.354662                     1.439590  0.161938   
408                   2.606591                     1.541049  0.211104   
244                   2.467280                     1.496453  0.107003   
416                   2.385520                     1.446058  0.148023   
428                   3.357158                     1.998352  0.220617   
429                   2.583783                     1.536578  0.150782   
446                   2.493814                     1.457018  0.137566   
452                   3.165592                     1.893741  0.163552   

     dream_team_appearances  
208                      76  
187                      72  
162                      38  
14                       26  
175                      18  
117                       3  
419                       3  
103                       1  
380                       0  
281                       0  
291                       0  
298                       0  
5                         0  
408                       0  
244                       0  
416                       0  
428                       0  
429                       0  
446                       0  
452                       0